In [2]:
import numpy as np
import pandas as pd

import sys
sys.path.append('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/libraries/')
from clinical import load_clinical
from data_processing import *

In [3]:
## Import the leiden clusters from the H&Es

clusters_frame_bioclavis = pd.read_csv('/mnt/cephfs/home/users/krakovic/sharedscratch/Histomorphological-Phenotype-Learning/results/BarlowTwins_3/LATTICeA_5x/h224_w224_n3_zdim128_filtered/rapids_2p5m/adatas/LATTICeA_BioclavisHE_4pc_5x_he_train_filtered_leiden_2p5__fold2.csv')
clusters_frame_gs = pd.read_csv('/mnt/cephfs/home/users/krakovic/sharedscratch/Histomorphological-Phenotype-Learning/results/BarlowTwins_3/LATTICeA_5x/h224_w224_n3_zdim128_filtered/rapids_2p5m/adatas/LATTICeA_GrandSlam_4pc_5x_he_combined_filtered_leiden_2p5__fold2.csv')
clusters_frame_gs = clusters_frame_gs.rename(columns={'combined_samples':'samples',
                                                      'combined_slides':'slides',
                                                      'combined_tiles':'tiles'})



clusters_frame_bioclavis['core_ID'] = clusters_frame_bioclavis['slides'].apply(lambda x: x.split('_')[2])
clusters_frame_bioclavis['TMA'] = clusters_frame_bioclavis['core_ID'].apply(lambda x: x.split('-')[0])

clusters_frame_gs['core_ID'] = clusters_frame_gs['slides'].apply(lambda x: x.split('_')[2])
clusters_frame_gs['TMA'] = clusters_frame_gs['core_ID'].apply(lambda x: x.split('-')[0])

clusters_frame_bioclavis = clusters_frame_bioclavis.loc[(clusters_frame_bioclavis['TMA'] == '12') | (clusters_frame_bioclavis['TMA'] == '13')]
clusters_frame_gs = clusters_frame_gs.loc[(clusters_frame_gs['TMA'] == '21') | (clusters_frame_gs['TMA'] == '22')]

clusters_frame = pd.concat([clusters_frame_bioclavis, clusters_frame_gs])
leiden_clusters = np.unique(clusters_frame['leiden_2.5'])
clusters_frame['core'] = clusters_frame['slides'].apply(lambda x: x[9:])

clusters_frame = clusters_frame.copy(deep=True)
clusters_frame

,Unnamed: 0,hist_subtype,labels,patterns,samples,slides,tiles,leiden_2.5,os_event_ind,os_event_data,core_ID,TMA,core
103,103.0,NaN,0.0,NaN,ACA_0670,ACA_0670_13-13-B,0_1.jpeg,35,0.0,126.115068,13-13-B,13,13-13-B
104,104.0,NaN,0.0,NaN,ACA_0670,ACA_0670_13-13-B,1_0.jpeg,35,0.0,126.115068,13-13-B,13,13-13-B
105,105.0,NaN,0.0,NaN,ACA_0670,ACA_0670_13-13-B,0_0.jpeg,35,0.0,126.115068,13-13-B,13,13-13-B
106,106.0,NaN,0.0,NaN,ACA_0670,ACA_0670_13-13-B,1_1.jpeg,35,0.0,126.115068,13-13-B,13,13-13-B
107,107.0,NaN,0.0,NaN,ACA_0670,ACA_0670_13-14-B,0_1.jpeg,46,0.0,126.115068,13-14-B,13,13-14-B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,NaN,NaN,NaN,NaN,ACA_1060,ACA_1060_22-09-H,1_1.jpeg,61,NaN,NaN,22-09-H,22,22-09-H
3050,NaN,NaN,NaN,NaN,ACA_0984,ACA_0984_21-03-A,0_0.jpeg,13,NaN,NaN,21-03-A,21,21-03-A
3051,NaN,NaN,NaN,NaN,ACA_0984,ACA_0984_21-03-A,0_1.jpeg,68,NaN,NaN,21-03-A,21,21-03-A
3052,NaN,NaN,NaN,NaN,ACA_0984,ACA_0984_21-03-A,1_0.jpeg,13,NaN,NaN,21-03-A,21,21-03-A


In [4]:
pivot_clusters_frame = pd.get_dummies(clusters_frame[['samples', 'core_ID', 'leiden_2.5']], prefix='cluster', columns=['leiden_2.5'])
pivot_clusters_frame = pivot_clusters_frame.groupby(by='core_ID').sum().reset_index()

leiden_clusters, malignant_clusters, stromal_clusters = load_topography()
malignant_clusters = list(malignant_clusters)+[10,16]

majority_clusters_frame = pivot_clusters_frame.copy(deep=True)

majority_clusters_frame = majority_clusters_frame[['core_ID']+[f'cluster_{col}' for col in malignant_clusters]]
majority_clusters_frame['max'] = majority_clusters_frame[[f'cluster_{col}' for col in malignant_clusters]].max(numeric_only=True, axis=1)
majority_clusters_frame['sum'] = majority_clusters_frame[[f'cluster_{col}' for col in malignant_clusters]].sum(numeric_only=True, axis=1)

cluster_pure_cores = majority_clusters_frame.loc[(majority_clusters_frame['max'] / majority_clusters_frame['sum']) > 0.5]
cluster_pure_cores['cluster'] = cluster_pure_cores[[f'cluster_{col}' for col in malignant_clusters]].idxmax(numeric_only=True, axis=1)
cluster_pure_cores['cluster'] = cluster_pure_cores['cluster'].apply(lambda x: 'HPC ' + str(x.split("_")[1]))
cluster_pure_cores['cluster'].value_counts()

/tmp/ipykernel_638868/4112667579.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivot_clusters_frame = pivot_clusters_frame.groupby(by='core_ID').sum().reset_index()
/tmp/ipykernel_638868/4112667579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_pure_cores['cluster'] = cluster_pure_cores[[f'cluster_{col}' for col in malignant_clusters]].idxmax(numeric_only=True, axis=1)
/tmp/ipykernel_638868/4112667579.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

HPC 52    19
HPC 11    14
HPC 44    13
HPC 64    13
HPC 50    13
HPC 35    12
HPC 41    12
HPC 27    11
HPC 16    10
HPC 47     9
HPC 46     8
HPC 13     7
HPC 67     7
HPC 48     6
HPC 17     6
HPC 21     6
HPC 28     6
HPC 68     5
HPC 69     5
HPC 5      5
HPC 54     5
HPC 0      5
HPC 10     5
HPC 61     3
HPC 34     3
HPC 31     2
HPC 33     2
HPC 30     2
HPC 40     2
Name: cluster, dtype: int64

In [5]:
cluster_pure_cores

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_61,cluster_64,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,cluster
0,12-02-A,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,2,2,HPC 27
1,12-02-B,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,HPC 0
2,12-02-C,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,3,4,HPC 67
3,12-02-D,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,2,2,HPC 10
6,12-02-H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,4,HPC 41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,22-14-A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,HPC 31
408,22-14-G,0,0,0,0,0,0,0,0,0,...,0,0,0,3,0,0,0,3,4,HPC 68
409,22-14-H,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,4,4,HPC 28
411,22-15-A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,HPC 50


In [6]:
def concat_phenotypes(row, colnames):
    positive_cols = [col for col in colnames if row[col] == 1]
    if len(positive_cols) == 0:
        return 'Negative'
    return '_'.join(['HPC ' + col.split("_")[1] for col in positive_cols])

# May take 'mixed' cases if the epithelial compartment is pure is. if the other HPC is a stromal one
colnames = [c for c in majority_clusters_frame.columns.values if c.startswith('cluster_')]
borderline_cores = majority_clusters_frame.loc[(majority_clusters_frame['max'] == 2)]
borderline_cores['hpc_phenotype'] = borderline_cores.apply(lambda row: concat_phenotypes(row, colnames), axis=1)
borderline_cores

/tmp/ipykernel_638868/893914929.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  borderline_cores['hpc_phenotype'] = borderline_cores.apply(lambda row: concat_phenotypes(row, colnames), axis=1)


,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_61,cluster_64,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hpc_phenotype
0,12-02-A,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,2,2,Negative
1,12-02-B,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,HPC 35
3,12-02-D,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,2,2,Negative
9,12-03-A,0,0,0,2,1,1,0,0,0,...,0,0,0,0,0,0,0,2,4,HPC 17_HPC 21
15,12-03-H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,HPC 41_HPC 44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,22-14-A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,HPC 50
405,22-14-D,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,2,4,Negative
410,22-14-I,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,HPC 13_HPC 50
411,22-15-A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,Negative


In [7]:
def check_companion_phenotype(row, malignant_clusters, stromal_clusters):
    clus1 = int(row['hpc_phenotype'].split("_")[0].split(" ")[1])
    clus2 = int(row['hpc_phenotype'].split("_")[1].split(" ")[1])


    if clus1 in malignant_clusters and clus2 in malignant_clusters:
        return -1
    
    elif clus1 in stromal_clusters and clus2 in malignant_clusters:
        return clus2
    
    elif clus1 in malignant_clusters and clus2 in stromal_clusters:
        return clus1
    
    else:
        return -1
    
borderline_cores = borderline_cores[borderline_cores['hpc_phenotype'] != 'Negative'] # filter those without hits
borderline_cores['single_phenotype'] = borderline_cores['hpc_phenotype'].apply(lambda x: 0 if '_' in x else 1)
borderline_cores = borderline_cores[borderline_cores['single_phenotype'] == 0]
borderline_cores['malignant_phenotype'] = borderline_cores.apply(lambda row: check_companion_phenotype(row, malignant_clusters, stromal_clusters), axis=1)
borderline_cores

/tmp/ipykernel_638868/2224595421.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  borderline_cores['single_phenotype'] = borderline_cores['hpc_phenotype'].apply(lambda x: 0 if '_' in x else 1)


,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_67,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hpc_phenotype,single_phenotype,malignant_phenotype
9,12-03-A,0,0,0,2,1,1,0,0,0,...,0,0,0,0,0,2,4,HPC 17_HPC 21,0,-1
15,12-03-H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,4,HPC 41_HPC 44,0,-1
17,12-03-J,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,2,4,HPC 30_HPC 68,0,-1
21,12-04-F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,4,HPC 50_HPC 54,0,-1
26,12-05-C,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,2,4,HPC 47_HPC 68,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,22-09-I,0,0,0,0,0,0,0,2,0,...,0,0,0,1,0,2,4,HPC 61_HPC 10,0,-1
382,22-10-A,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,2,4,HPC 35_HPC 46,0,-1
388,22-10-I,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,2,4,HPC 30_HPC 48,0,-1
391,22-11-E,0,0,1,0,0,0,0,0,0,...,0,0,2,0,0,2,4,HPC 11_HPC 41,0,-1


In [8]:
# Revised classification -- see notes for details on rationale
def assign_supercluster(x):
    if x in ['HPC 67', 'HPC 0', 'HPC 50', 'HPC 44']: # n=4
        return 'Hot, cohesive'
    elif x in ['HPC 10', 'HPC 16', 'HPC 28', 'HPC 61', 'HPC 69', 'HPC 48', 'HPC 13']: # n=7
        return 'Hot, discohesive'
    elif x in ['HPC 33', 'HPC 21', 'HPC 47', 'HPC 17', 'HPC 11', 'HPC 68', 'HPC 34']: # n=7
        return 'Cold, cohesive'
    elif x in ['HPC 30', 'HPC 40', 'HPC 27', 'HPC 64', 'HPC 5', 'HPC 31']: # n=6
        return 'Cold, discohesive'
    else:
        return 'Unknown'
    
def count_hot_cohesive(row): # n=4
    return row['cluster_67'] + row['cluster_0'] + row['cluster_50'] + row['cluster_44']

def count_hot_discohesive(row): # n=7
    return row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_13'] + row['cluster_69']
    # return row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_13'] + row['cluster_5'] + row['cluster_35'] + row['cluster_31']

def count_cold_cohesive(row): # n=7
    return row['cluster_33'] + row['cluster_21'] + row['cluster_47'] + row['cluster_17'] + row['cluster_11'] + row['cluster_68'] + row['cluster_34']

def count_cold_discohesive(row): # n=6
    return row['cluster_30'] + row['cluster_40'] + row['cluster_27'] + row['cluster_64'] + row['cluster_5'] + row['cluster_31']

majority_clusters_frame['hot_cohesive'] = majority_clusters_frame.apply(lambda row: count_hot_cohesive(row), axis=1)
majority_clusters_frame['hot_discohesive'] = majority_clusters_frame.apply(lambda row: count_hot_discohesive(row), axis=1)
majority_clusters_frame['cold_cohesive'] = majority_clusters_frame.apply(lambda row: count_cold_cohesive(row), axis=1)
majority_clusters_frame['cold_discohesive'] = majority_clusters_frame.apply(lambda row: count_cold_discohesive(row), axis=1)


In [9]:
majority_clusters_frame

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_68,cluster_69,cluster_10,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive
0,12-02-A,0,0,0,0,0,0,2,0,0,...,0,0,0,0,2,2,0,0,0,2
1,12-02-B,2,0,0,0,0,0,0,0,0,...,0,0,0,0,2,3,2,0,0,0
2,12-02-C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,4,3,0,0,0
3,12-02-D,0,0,0,0,0,0,0,0,0,...,0,0,2,0,2,2,0,2,0,0
4,12-02-F,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,2,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,22-15-B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
413,22-15-D,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,2,0,0,1,1
414,22-15-E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,22-15-I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,2,0,0,0


In [10]:
majority_clusters_frame['sc_max'] = majority_clusters_frame[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].max(numeric_only=True, axis=1)
majority_clusters_frame

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_69,cluster_10,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive,sc_max
0,12-02-A,0,0,0,0,0,0,2,0,0,...,0,0,0,2,2,0,0,0,2,2
1,12-02-B,2,0,0,0,0,0,0,0,0,...,0,0,0,2,3,2,0,0,0,2
2,12-02-C,0,0,0,0,0,0,0,0,0,...,0,0,0,3,4,3,0,0,0,3
3,12-02-D,0,0,0,0,0,0,0,0,0,...,0,2,0,2,2,0,2,0,0,2
4,12-02-F,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,0,0,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,22-15-B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
413,22-15-D,0,0,0,0,1,0,0,0,0,...,0,0,0,1,2,0,0,1,1,1
414,22-15-E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,22-15-I,0,0,0,0,0,0,0,0,0,...,0,0,0,2,2,2,0,0,0,2


In [11]:
supercluster_pure_cores = majority_clusters_frame.loc[(majority_clusters_frame['sc_max'] / majority_clusters_frame['sum']) > 0.5]
supercluster_pure_cores['supercluster'] = supercluster_pure_cores[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].idxmax(numeric_only=True, axis=1)
# supercluster_pure_cores['supercluster'] = supercluster_pure_cores['supercluster'].apply(lambda x: x.split("_")[0].capitalize())
supercluster_pure_cores['supercluster'].value_counts()

/tmp/ipykernel_638868/785160789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supercluster_pure_cores['supercluster'] = supercluster_pure_cores[['hot_cohesive', 'hot_discohesive', 'cold_cohesive', 'cold_discohesive']].idxmax(numeric_only=True, axis=1)


hot_discohesive     65
cold_cohesive       59
cold_discohesive    51
hot_cohesive        51
Name: supercluster, dtype: int64

In [12]:
supercluster_pure_cores = supercluster_pure_cores.merge(cluster_pure_cores[['core_ID', 'cluster']], on='core_ID', how='left')

In [13]:
# supercluster_pure_cores[['core_ID', 'supercluster', 'cluster']].to_csv('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/final_figures/codex/codex_pure_superclusters.csv')

In [14]:
supercluster_pure_cores[['core_ID', 'supercluster', 'cluster']]

,core_ID,supercluster,cluster
0,12-02-A,cold_discohesive,HPC 27
1,12-02-B,hot_cohesive,HPC 0
2,12-02-C,hot_cohesive,HPC 67
3,12-02-D,hot_discohesive,HPC 10
4,12-02-F,cold_cohesive,NaN
...,...,...,...
221,22-14-A,cold_discohesive,HPC 31
222,22-14-G,cold_cohesive,HPC 68
223,22-14-H,hot_discohesive,HPC 28
224,22-15-A,hot_cohesive,HPC 50


In [15]:
supercluster_pure_cores_HD_split = supercluster_pure_cores.copy(deep=True)
supercluster_pure_cores_HD_split['highrisk_HD'] = supercluster_pure_cores_HD_split.apply(lambda row: row['cluster_13'], axis=1)
supercluster_pure_cores_HD_split['lowrisk_HD'] = supercluster_pure_cores_HD_split.apply(lambda row: row['cluster_10'] + row['cluster_16'] + row['cluster_28'] + row['cluster_61'] + row['cluster_48'] + row['cluster_69'], axis=1)
supercluster_pure_cores_HD_split['consensus_HD_equal'] = supercluster_pure_cores_HD_split.apply(lambda row: 1 if row['highrisk_HD'] >= row['lowrisk_HD'] else 0, axis=1)
supercluster_pure_cores_HD_split['consensus_HD_more'] = supercluster_pure_cores_HD_split.apply(lambda row: 1 if row['highrisk_HD'] > row['lowrisk_HD'] else 0, axis=1)
supercluster_pure_cores_HD_split

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,hot_discohesive,cold_cohesive,cold_discohesive,sc_max,supercluster,cluster,highrisk_HD,lowrisk_HD,consensus_HD_equal,consensus_HD_more
0,12-02-A,0,0,0,0,0,0,2,0,0,...,0,0,2,2,cold_discohesive,HPC 27,0,0,1,0
1,12-02-B,2,0,0,0,0,0,0,0,0,...,0,0,0,2,hot_cohesive,HPC 0,0,0,1,0
2,12-02-C,0,0,0,0,0,0,0,0,0,...,0,0,0,3,hot_cohesive,HPC 67,0,0,1,0
3,12-02-D,0,0,0,0,0,0,0,0,0,...,2,0,0,2,hot_discohesive,HPC 10,0,2,0,0
4,12-02-F,0,0,0,0,0,0,0,0,0,...,0,2,0,2,cold_cohesive,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,22-14-A,0,0,0,0,0,0,0,0,0,...,0,0,2,2,cold_discohesive,HPC 31,0,0,1,0
222,22-14-G,0,0,0,0,0,0,0,0,0,...,0,3,1,3,cold_cohesive,HPC 68,0,0,1,0
223,22-14-H,0,0,0,0,0,0,0,4,0,...,4,0,0,4,hot_discohesive,HPC 28,0,4,0,0
224,22-15-A,0,0,0,0,0,0,0,0,0,...,0,0,0,2,hot_cohesive,HPC 50,0,0,1,0


In [16]:
supercluster_pure_cores_HD_split[supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive']['consensus_HD_equal'].value_counts()

0    57
1     8
Name: consensus_HD_equal, dtype: int64

In [17]:
supercluster_pure_cores_HD_split[supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive']['consensus_HD_more'].value_counts()

0    58
1     7
Name: consensus_HD_more, dtype: int64

In [18]:
# supercluster_pure_cores_HD_split[['core_ID', 'supercluster', 'cluster', 'consensus_HD_equal', 'consensus_HD_more']].to_csv('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/final_figures/codex/codex_pure_superclusters_split_HPC13.csv')

### For GeoMx

In [19]:
geomx_cases = pd.read_csv('/mnt/cephfs/sharedscratch/users/krakovic/notebooks/geomx_gs/gs_pure_cores_nicolas.csv', index_col=0)
geomx_cases

,core_ID,cluster,inGeoMx,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,21-02-A,HPC 46,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21-02-B,HPC 17,False,NaN,NaN,HPC_11,HPC_13,HPC_16,HPC_17,HPC_21,...,HPC_47,HPC_48,HPC_5,HPC_50,HPC_52,HPC_54,HPC_61,HPC_64,HPC_67,HPC_69
2,21-02-C,HPC 61,False,NaN,NaN,3,1,2,5,4,...,2,6,1,3,6,3,1,3,4,1
3,21-02-D,HPC 52,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21-02-E,HPC 0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,22-14-I,HPC 40,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,22-15-A,HPC 50,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,22-15-D,HPC 17,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,22-15-I,HPC 50,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
inGeomx_cores = geomx_cases[geomx_cases['inGeoMx'] == True]['core_ID'].values

supercluster_pure_cores[supercluster_pure_cores['core_ID'].isin(inGeomx_cores)]['supercluster'].value_counts()

hot_discohesive     12
hot_cohesive        11
cold_cohesive        9
cold_discohesive     7
Name: supercluster, dtype: int64

In [21]:
supercluster_pure_cores[supercluster_pure_cores['core_ID'].isin(inGeomx_cores)][['core_ID', 'supercluster', 'cluster']]

,core_ID,supercluster,cluster
111,21-03-B,hot_discohesive,NaN
118,21-04-C,cold_discohesive,HPC 27
120,21-04-F,cold_discohesive,HPC 40
122,21-04-J,hot_cohesive,HPC 50
124,21-05-E,cold_cohesive,HPC 11
126,21-05-G,cold_cohesive,HPC 21
127,21-05-H,hot_discohesive,HPC 61
128,21-05-J,cold_cohesive,NaN
137,21-07-C,cold_cohesive,NaN
139,21-07-F,cold_cohesive,HPC 47


In [22]:
supercluster_pure_cores_HD_split[(supercluster_pure_cores_HD_split['core_ID'].isin(inGeomx_cores)) & (supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive')]['consensus_HD_more'].value_counts()

0    11
1     1
Name: consensus_HD_more, dtype: int64

In [23]:
supercluster_pure_cores_HD_split[(supercluster_pure_cores_HD_split['core_ID'].isin(inGeomx_cores)) & (supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive')]['consensus_HD_equal'].value_counts()

0    11
1     1
Name: consensus_HD_equal, dtype: int64

In [25]:
supercluster_pure_cores_HD_split[(supercluster_pure_cores_HD_split['core_ID'].isin(inGeomx_cores)) & (supercluster_pure_cores_HD_split['supercluster'] == 'hot_discohesive') & (supercluster_pure_cores_HD_split['consensus_HD_more'] == 1)]

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,hot_discohesive,cold_cohesive,cold_discohesive,sc_max,supercluster,cluster,highrisk_HD,lowrisk_HD,consensus_HD_equal,consensus_HD_more
202,22-07-H,0,0,0,3,1,0,0,0,0,...,3,1,0,3,hot_discohesive,HPC 13,3,0,1,1


In [30]:
for_geomx = supercluster_pure_cores[supercluster_pure_cores['core_ID'].isin(inGeomx_cores)]
for_geomx = for_geomx[for_geomx['core_ID'] != '22-07-H']
for_geomx

,core_ID,cluster_0,cluster_5,cluster_11,cluster_13,cluster_17,cluster_21,cluster_27,cluster_28,cluster_30,...,cluster_16,max,sum,hot_cohesive,hot_discohesive,cold_cohesive,cold_discohesive,sc_max,supercluster,cluster
111,21-03-B,0,0,0,0,0,0,0,0,0,...,1,1,3,0,2,0,0,2,hot_discohesive,NaN
118,21-04-C,0,0,0,0,0,0,4,0,0,...,0,4,4,0,0,0,4,4,cold_discohesive,HPC 27
120,21-04-F,0,0,0,0,0,0,0,0,0,...,0,2,3,0,1,0,2,2,cold_discohesive,HPC 40
122,21-04-J,0,0,0,0,0,1,0,0,0,...,0,3,4,3,0,1,0,3,hot_cohesive,HPC 50
124,21-05-E,0,0,2,0,0,0,0,0,0,...,0,2,3,1,0,2,0,2,cold_cohesive,HPC 11
126,21-05-G,0,0,0,0,0,4,0,0,0,...,0,4,4,0,0,4,0,4,cold_cohesive,HPC 21
127,21-05-H,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,0,1,hot_discohesive,HPC 61
128,21-05-J,0,0,0,0,2,0,0,0,0,...,0,2,4,0,0,3,1,3,cold_cohesive,NaN
137,21-07-C,0,0,0,0,2,0,0,0,0,...,0,2,4,0,0,4,0,4,cold_cohesive,NaN
139,21-07-F,0,0,0,0,0,0,0,1,0,...,0,3,4,0,1,3,0,3,cold_cohesive,HPC 47


In [32]:
for_geomx[for_geomx['core_ID'].isin(inGeomx_cores)][['core_ID', 'supercluster', 'cluster']].to_csv('/nfs/home/users/krakovic/sharedscratch/notebooks/latticea_he/final_figures/inGeoMx_supercluster_pure_cores.csv')

In [30]:
supercluster_pure_cores[supercluster_pure_cores['core_ID'].isin(inGeomx_cores)][['core_ID', 'supercluster', 'cluster']]

,core_ID,supercluster,cluster
111,21-03-B,hot_discohesive,NaN
118,21-04-C,cold_discohesive,HPC 27
120,21-04-F,cold_discohesive,HPC 40
122,21-04-J,hot_cohesive,HPC 50
124,21-05-E,cold_cohesive,HPC 11
126,21-05-G,cold_cohesive,HPC 21
127,21-05-H,hot_discohesive,HPC 61
128,21-05-J,cold_cohesive,NaN
137,21-07-C,cold_cohesive,NaN
139,21-07-F,cold_cohesive,HPC 47
